In [1]:
import pandas as pd
import numpy as np


In [2]:
myFile = pd.read_csv('D:/project/550_Pattern/Project_1/550-p1-cset-krk-1.csv')
myFile
myFrame=pd.DataFrame(myFile.values, columns = ["White-King-File", "White-King-rank", "White-Rook-file", "White-Rook-rank","Black-King-file","Black-King-rank","Class"])
myFrame

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
1,b,1,d,7,c,7,draw
2,d,2,g,1,g,2,draw
3,d,4,g,6,f,6,draw
4,c,1,b,5,a,5,draw
...,...,...,...,...,...,...,...
214,d,3,d,7,a,7,nine
215,b,1,b,4,g,1,nine
216,d,4,e,8,g,2,nine
217,c,2,d,5,f,1,nine


In [3]:
myFrame.shape

(219, 7)

In [4]:
# Split into training set:60%, holdout set:20%, Validation set: 20%

training_set = myFrame.sample(frac=0.6)
remaining_set = myFrame.drop(training_set.index)
training_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
77,c,3,c,8,b,1,three
206,d,4,f,8,g,6,nine
187,d,2,b,5,h,2,eight
22,c,2,a,8,a,2,zero
64,d,3,a,4,d,1,two
...,...,...,...,...,...,...,...
205,d,3,e,6,h,3,nine
43,c,2,d,4,a,3,one
30,c,2,d,7,a,1,one
3,d,4,g,6,f,6,draw


In [5]:
remaining_set


,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
1,b,1,d,7,c,7,draw
2,d,2,g,1,g,2,draw
8,b,1,h,3,g,2,draw
10,d,3,h,4,h,3,draw
12,d,4,e,1,d,1,draw
...,...,...,...,...,...,...,...
212,c,3,c,7,h,2,nine
213,d,4,d,8,g,1,nine
216,d,4,e,8,g,2,nine
217,c,2,d,5,f,1,nine


In [6]:
classes, cls_counts = np.unique(training_set['Class'], return_counts = True)
classes, cls_counts


(array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
        'three', 'two', 'zero'], dtype=object),
 array([11, 13, 12, 13, 12, 13, 16, 12, 10, 17,  2]))

In [7]:
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [8]:
calculate_entropy(training_set['Class'])

3.3682650161282606

In [9]:

for cols in training_set.columns[:-1]:
    print(cols)

White-King-File
White-King-rank
White-Rook-file
White-Rook-rank
Black-King-file
Black-King-rank


In [10]:
# dataset = training_set
# feature = Feature or attribute of the dataset, i.e column name (training_set.columns[:-1]) E.g White-King-File
# label = Name of Class label colums, i.e. 'Class'

def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy    
    return feature_info_gain

In [13]:
calculate_information_gain(training_set, 'White-King-File', 'Class')

0.498931550891192

In [29]:
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label], return_counts=True)
    unique_data = np.unique(dataset[label])
    print('datum',datum)
    print('unique_data',unique_data)
    if len(unique_data)<=1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]
    elif len(features) == 0:
        return parent
    else:
        parent = unique_data[np.argmax(datum[1])]
        print('datum[1]',datum[1])
        print("parent",parent)
        
        item_values =  [calculate_information_gain(dataset, feature, label) for feature in features]
        optimal_feature_index = np.argmax(item_values)
        optimal_feature = features[optimal_feature_index]
        decision_tree = {optimal_feature:{}}
        features = [i for i in features if i!=optimal_feature]
        
        print(item_values)
        for value in np.unique(dataset[optimal_feature]):
            min_data = dataset.where(dataset[optimal_feature] == value).dropna()
            
            min_tree = create_decision_tree(min_data, df, features, label, parent)
            
            decision_tree[optimal_feature][value] = min_tree
        
        return (decision_tree)
        

In [30]:
features = training_set.columns[:-1]
label = 'Class'
parent=None
features

Index(['White-King-File', 'White-King-rank', 'White-Rook-file',
       'White-Rook-rank', 'Black-King-file', 'Black-King-rank'],
      dtype='object')

In [31]:
decision_tree = create_decision_tree(training_set,training_set,features,label,parent)
decision_tree


datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([11, 13, 12, 13, 12, 13, 16, 12, 10, 17,  2]))
unique_data ['draw' 'eight' 'five' 'four' 'nine' 'one' 'seven' 'six' 'three' 'two'
 'zero']
datum[1] [11 13 12 13 12 13 16 12 10 17  2]
parent two
[0.498931550891192, 0.329720032142756, 0.5793138635869881, 0.5276930214093714, 0.7500867880528492, 0.5949785620695485]
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([11, 13, 12, 13, 12, 13, 16, 12, 10, 17,  2]))
unique_data ['draw' 'eight' 'five' 'four' 'nine' 'one' 'seven' 'six' 'three' 'two'
 'zero']
datum[1] [11 13 12 13 12 13 16 12 10 17  2]
parent two
[0.5816396216295514, 0.5030075057928696, 1.067240012190518, 1.2098030670731785, 1.1793777211652663]
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array(

IndexError: index 9 is out of bounds for axis 0 with size 2